# NVIDIA Meeting Recorder Agent

In this notebook, we'll build a meeting recorder agent capable of finding appointments on your Google Calendar, recording the meetings, and sending the recording assets — including video, transcription, and summary — via email.

By the end of the notebook, you'll have an agent that can perform several key tasks, such as:

- Checking your meeting schedule on Google Calendar and retrieving relevant meeting information  
- Creating a meeting recorder to capture the session and monitoring its real-time status  
- Sending the recording, transcription, and summary of the meeting to a specified email address

Along the way, we’ll explore how to persist the agent’s memory using the xpander SDK, and observe the agent’s real-time thought process as it completes each task.

We’ll be using the `xpander.ai` Python SDK and NVIDIA NIM to bring this agent to life. The xpander SDK manages the workflow and orchestration, while NVIDIA NIM powers the agent's large language model.

<div style="height:300px; padding:10px;">

## Table of Contents
<ul>
  <li><a href="#Fetch-xpander.ai-API-Key">Fetch xpander.ai API Key</a></li>
  <li><a href="#Fetch-NVIDIA-NIM-API-Key">Fetch NVIDIA NIM API Key</a></li>
  <li><a href="#Setup-and-Build-the-AI-Agent">Setup and Build the AI Agent</a></li>
  <li><a href="#Interact-and-Execute-Agent">Interact and Execute Agent</a>
    <ul>
      <li><a href="#Define-the-Meeting-Agent-Class">Define the Meeting Agent Class</a></li>
      <li><a href="#Initialize-the-Meeting-Agent">Initialize the Meeting Agent</a></li>
      <li><a href="#Example-Use-Case-1:-Test-Agent's-Capabilities">Example Use Case 1: Test Agent's Capabilities</a></li>
      <li><a href="#Example-Use-Case-2:-Retrieve-Upcoming-Calendar-Events">Example Use Case 2: Retrieve Upcoming Calendar Events</a></li>
      <li><a href="#Example-Use-Case-3:-Set-Up-Meeting-Recorder">Example Use Case 3: Set Up Meeting Recorder</a></li>
      <li><a href="#Example-Use-Case-4:-Check-the-Meeting-Recorder-Status">Example Use Case 4: Check the Meeting Recorder Status</a></li>
      <li><a href="#Example-Use-Case-5:-Send-Recording-Assets-via-Email">Example Use Case 5: Send Recording Assets via Email</a></li>
    </ul>
  </li>
  <li><a href="#Summary:-Working-with-the-Meeting-Recorder-Agent">Summary: Working with the Meeting Recorder Agent</a></li>
  <li><a href="#Next-Steps">Next Steps</a></li>
</ul>

</div>

Since we're going to use xpander and NVIDIA NIM, then we need to set up two different API keys before we start: xpander API key and NVIDIA NIM API key. Let's fetch our xpander.ai API key first.

<a id="Fetch-xpander.ai-API-Key"></a>
## Fetch xpander.ai API Key

The simplest way to fetch your `xpander.ai` API key is by using the xpander CLI.

To install the CLI, you'll need to have Node.js installed on your system. If you haven’t installed Node.js yet, visit their [download page](https://nodejs.org/en/download) and follow the installation instructions.

Once Node.js is installed, run the following command to install the xpander CLI globally:

```
npm install -g xpander-cli
```

Next, log in to your xpander account via the CLI to retrieve your API key by running:

```
xpander login
```

Your API key will be displayed in the terminal, as shown in the visualization below:

![img_1]

Copy your xpander API key and paste it into the `.env` file you created after completing the steps in the **Getting Started** section of this repo.


<a id="Fetch-NVIDIA-NIM-API-Key"></a>
## Fetch NVIDIA NIM API Key

You can get the NVIDIA NIM API key for free. Just head over to [https://build.nvidia.com/models](https://build.nvidia.com/models) and select the LLM you’d like to use.  

For this blueprint, we’ll use NVIDIA’s `llama-3.1-nemotron-ultra-253b-v1`.

![img_2]

Click on **Get API Key** in the top-right corner, then copy your NVIDIA NIM API key and paste it into the `.env` file you created after completing the steps in the **Getting Started** section of this repo.


<a id="Setup-and-Build-the-AI-Agent"></a>
## Set Up and Build the AI Agent

Now that we've set up the required credentials, we can start building our meeting recorder agent.

We've prepared a template that you can use to instantly spin up the agent. Just click the button below to build the agent using our template.

<div style="text-align: center;">
    <a href="https://app.xpander.ai/templates/383f7607-0223-42a8-a472-5c7b1fdf6315" target="_blank">
        <button style="
            background-color: #6741d9;
            color:white;
            padding: 10px 20px;
            font-size: 14px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
    ">
            Build AI Agent with xpander Template
        </button>
    </a>
</div>

After clicking the button, you'll be redirected to the xpander console, as shown in the visualization below:

![Img 3]

Using a template to build an AI agent is straightforward. The template handles everything needed to set up the agent:

- It configures the required interface(s), such as Google Calendar, along with the relevant operations (e.g., retrieve events, create events, delete events, etc.).
- It orchestrates the agent’s logical workflow, if necessary.

![Img 4]

Once you click **Finish**, you’ll see that a new AI agent called **AI Meeting Recorder Agent (Imported)** has been successfully created in your xpander account.

![Img 5]

Clicking on the newly created agent will show you the agent's orchestration derived from the template, along with the configured interfaces needed to run the agent.

Since a key feature of this agent is the ability to check your Google Calendar for scheduled meetings, you’ll need to grant xpander permission to access your calendar.  

If you haven’t granted this permission yet, you’ll see a warning icon next to the Google Calendar operation, as shown in the visualization below:

![Img 6]

To grant xpander permission to access our Google Calendar, click on the warning icon. Next, click **Sign In with Google Calendar** and then follow the prompts.

Once you grant the permission to xpander, then the warning icon should dissapear.

The last thing we need to do is to bind the agent ID with our API key so that we can interact with the agent. Additionally, we also need to store the agent ID into the `.env` file. To do these things, click on **SDK** node at the top of the graph visualization of your xpander console, as you can see below:

![Img 7]

Copy the agent ID into the `.env` file, and then click on **Assign API Keys** and tick on the box next to your xpander API key as can be seen below:

![Img 8]

And now we're ready to interact with our agent!

<a id="Interact-and-Execute-Agent"></a>
## Interact and Execute Agent

In thi section,  we’ll execute and interact with the agent we created with the template. As mentioned in the introduction of this notebook, we’ll be using the agent to perform several tasks, such as:

- Checking our meeting schedule on Google Calendar and retrieving relevant meeting information

- Creating a meeting recorder to capture the session and checking its real-time status

- Sending the recording, transcription, and summary of the meeting to a specified email address


First, let’s retrieve all the necessary credentials, such as the xpander API key, NVIDIA NIM API key, and the agent ID we created in the previous notebook.



In [1]:
import os
import datetime
import time
import re
import ast
import random
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, ToolCall, ToolCallType, Tokens, LLMTokens, Agent
from openai import OpenAI

# Load environment variables
load_dotenv()

# Verify required API keys are available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
agent_id = os.environ.get("XPANDER_AGENT_ID")
nvidia_nim_api_key = os.environ.get("NVIDIA_NIM_API_KEY")


if not all([xpander_api_key, nvidia_nim_api_key, agent_id]):
    missing = []
    if not xpander_api_key: missing.append("XPANDER_API_KEY")
    if not nvidia_nim_api_key: missing.append("NVIDIA_NIM_API_KEY")
    if not agent_id: missing.append("XPANDER_AGENT_ID")
    print(f"❗️ Missing required environment variables: {', '.join(missing)}")
else:
    print(f"✅ All required API keys loaded successfully")
    
print(f"Open the agent alogside this notebook at https://app.xpander.ai/agents/{agent_id}")

✅ All required API keys loaded successfully
Open the agent alogside this notebook at https://app.xpander.ai/agents/8445dd7e-167e-49f8-9ecf-8361b22f92d9


<a id="Define-the-Meeting-Agent-Class"></a>
### Define the Meeting Agent Class

Now, let’s create our `MeetingAgent` class. This class will handle both the initialization of the agent and the logic for processing input prompts and generating responses.

For each prompt the agent receives, it processes the input and generates a response through a looped operation. Each iteration of the loop consists of the following steps:

- The LLM generates a response based on the current state.

- It evaluates whether it can complete the task directly or if it needs to call a tool (e.g., Google Calendar, xpander built-in actions, etc.).

- If a tool is needed, the agent selects the appropriate tool and provides the required input arguments.

- xpander executes the tool call and returns the result to the agent for the next iteration.

- Once the LLM determines that it has all the necessary information to answer the user’s query, it calls the `xpfinish-agent-execution-finished` function, essentially ending the loop.

In [2]:
def get_function_and_payload(model_response):
    """Function to parse tool name and its payload"""

    match = re.search(r"\[([A-Za-z0-9_\-]+)({.*})\]", model_response["choices"][0]["message"]["content"])

    if match:
        tool_name = match.group(1)  
        payload_str = match.group(2) 
        payload = ast.literal_eval(payload_str)  
    else:
        tool_name = None
        payload = None
    
    
    return tool_name, payload

In [3]:
class MeetingAgent:
    """Class for running the meeting recorder agent"""
    
    def __init__(self, nvidia_nim_api_key, xpander_api_key, agent_id):
        self.agent_id = agent_id
        self.xpander_client = XpanderClient(api_key=xpander_api_key)
        self.openai_client =  OpenAI(
            base_url = "https://integrate.api.nvidia.com/v1",
            api_key = nvidia_nim_api_key
            )
 
        self.agent : Agent = self.xpander_client.agents.get(agent_id=self.agent_id)


    def run(self, prompt=None, thread_id=None):
        """Run the agent with the given prompt"""

        is_finished = False
        task = prompt or f"Please check the status of all recorded meetings."

        print("-" * 60)
        print(f"{'Continuing conversation in thread: ' + thread_id if thread_id else 'Starting a new conversation thread'}")
        print(f"Processing Task: \n\n{task}\n\n")
        print("-" * 60)
           
        # Create task with or without thread_id
        self.agent.add_task(input=task, thread_id=thread_id if thread_id else None)
        self.agent.add_messages([{"role": "assistant", "content": "the time now is : " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}])

        # Initialize token tracking and timing
        execution_tokens = Tokens(worker=LLMTokens(completion_tokens=0, prompt_tokens=0, total_tokens=0))
        execution_start_time = time.perf_counter()
          
        # Run the agent until it's finished
        while not is_finished:

            # Track start time for this inference
            start_time = time.perf_counter()

            # Get response from OpenAI and process
            response = self.openai_client.chat.completions.create(
                model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
                messages= self.agent.messages,
                temperature=0.0

            )

            # Track token usage
            execution_tokens.worker.completion_tokens += response.usage.completion_tokens
            execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
            execution_tokens.worker.total_tokens += response.usage.total_tokens

                
            # Report LLM usage to Xpander
            self.agent.report_llm_usage(
                llm_response=response.model_dump(),
                llm_inference_duration=time.perf_counter() - start_time,
                llm_provider=LLMProvider.NVIDIA_NIM
            )

            self.agent.add_messages(response.model_dump())

            # Extract and execute any tool calls
            tool_name, payload = get_function_and_payload(response.model_dump())

            tool_calls = [ToolCall(
                            name=tool_name,
                            type=ToolCallType.XPANDER,
                            payload=payload,
                            tool_call_id=f"call_{random.randint(100000, 999999)}"
                        )]
            
            for tool_call in tool_calls:
                print("AI decided to call : ", tool_call.name)
                print("With the following payload : ", tool_call.payload)

            
            if tool_calls:
                print("Executing tool calls")
                responses = self.agent.run_tools(tool_calls=tool_calls)
                for response in responses:
                    print("Tool call function name : ", response.function_name)
                    print("HTTP Status code : ", response.status_code) 
                    if str(response.function_name) == "xpfinish-agent-execution-finished":
                        is_finished = True
        
            if is_finished:
                break

        # Process results
        execution_end_time = time.perf_counter()
        result = self.agent.retrieve_execution_result()
        thread_id = result.memory_thread_id
        print(f"Your thread ID is: {thread_id}")
        result_text = result.result
    
        # Report execution metrics to Xpander
        self.agent.report_execution_metrics(
            llm_tokens=execution_tokens,
            ai_model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
        )
        print(f"Status: {result.status}")
        print(f"Result: {result_text}")
        print(f"Execution duration: {execution_end_time - execution_start_time:.2f} seconds")
    
        # Return both the result text and thread ID
        return result_text, thread_id

<a id="Initialize-the-Meeting-Agent"></a>
### Initialize the Meeting Agent

Now that we’ve defined the logic for how the agent should process user prompts, let’s go ahead and initialize our meeting agent using the `MeetingAgent` class we just created.

We also need to provide the agent with clear instructions, roles, and specific goals. This ensures that:

- It understands its task and objective

- It follows the correct workflow when handling user requests

- It knows which tools to use in various situations to effectively complete a task

In [4]:
# Initialize the meeting agent
meeting_agent = MeetingAgent(
    nvidia_nim_api_key=nvidia_nim_api_key,
    xpander_api_key=xpander_api_key,
    agent_id=agent_id
)

meeting_agent.agent.instructions.general = f"""
Reasoning mode: ON
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
Based on any given question from the user, you shouldn't call the same function/tool more than once.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of <TOOLCALL>[func_name1{{'params_name1': 'params_value1', 'params_name2': 'params_value2'...}}]</TOOLCALL>

You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke.

<AVAILABLE_TOOLS>{meeting_agent.agent.get_tools(llm_provider=LLMProvider.NVIDIA_NIM)}</AVAILABLE_TOOLS>
"""

meeting_agent.agent.instructions.role = """
You are a helpful meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.
"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id.
"""

b'{"stdout":"bG9hZGluZyBhZ2VudCA4NDQ1ZGQ3ZS0xNjdlLTQ5ZjgtOWVjZi04MzYxYjIyZjkyZDkK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCA4NDQ1ZGQ3ZS0xNjdlLTQ5ZjgtOWVjZi04MzYxYjIyZjkyZDkgd2l0aCBleGVjdXRpb24gMGI4NGRlNGQtZDhjNi00MDM5LWFjM2ItYTU2MTg0ZGQ4NjlhCg=="}\n'
b'{"stdout":"cnVubmluZyB0b29sIENhbGVuZGFyRXZlbnRNYW5hZ2VtZW50R2V0Q2FsZW5kYXJFdmVudHNCeUlkIG9uIGFnZW50IDg0NDVkZDdlLTE2N2UtNDlmOC05ZWNmLTgzNjFiMjJmOTJkOSB3aXRoIGV4ZWN1dGlvbiAxZGZiOWY0Ny0wNmNhLTQ4NGQtYjkzNy0yYTc4M2Y5OTUwNjIK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCA4NDQ1ZGQ3ZS0xNjdlLTQ5ZjgtOWVjZi04MzYxYjIyZjkyZDkgd2l0aCBleGVjdXRpb24gMWRmYjlmNDctMDZjYS00ODRkLWI5MzctMmE3ODNmOTk1MDYyCg=="}\n'


<a id="Example-Use-Case-1:-Test-Agent's-Capabilities"></a>
### Example Use Case 1: Test Agent's Capabilities

Once we have setup everything, let's test the agent capabilities by using the following prompt:

In [5]:
prompt = """
Hi! What can you do?
"""

result, thread_id = meeting_agent.run(prompt=prompt)

------------------------------------------------------------
Starting a new conversation thread
Processing Task: 


Hi! What can you do?



------------------------------------------------------------
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'I can help you record meetings, check the status of recording tools, and retrieve meeting assets using the recording bot ID. Please let me know how I can assist you today!', 'is_success': True}}
Executing tool calls
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: b182539a-e26d-4d55-826e-475130bf56ed
Status: ExecutionStatus.COMPLETED
Result: I can help you record meetings, check the status of recording tools, and retrieve meeting assets using the recording bot ID. Please let me know how I can assist you today!
Execution duration: 6.82 seconds


After completing the task, xpander generates a thread ID for the agent's run. This thread ID is crucial for maintaining conversation history. It allows the agent to understand context and handle follow-up questions related to a previous query. We can simply include the same thread ID in subsequent prompts.

<a id="Example-Use-Case-2:-Retrieve-Upcoming-Calendar-Events"></a>
### Example Use Case 2: Retrieve Upcoming Calendar Events

Since our agent now has access to Google Calendar, we can ask it to retrieve upcoming meetings from our schedule. To execute the agent with memory persistence, use the thread ID retrieved from the previous interaction and pass it as an argument along with your prompt.

In [6]:
# Use the same thread_id to maintain context from the authorization
prompt = """
Please list my upcoming meetings for the next 3 days. 
For each meeting, include the title, date, time, and participants if available.
"""

meeting_agent.run(prompt=prompt,  thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: b182539a-e26d-4d55-826e-475130bf56ed
Processing Task: 


Please list my upcoming meetings for the next 3 days. 
For each meeting, include the title, date, time, and participants if available.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'timeMin': '2025-05-10T20:33:06Z', 'timeMax': '2025-05-13T20:33:06Z', 'maxResults': 10, 'singleEvents': True, 'orderBy': 'startTime'}, 'pathParams': {'calendarId': 'primary'}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'Here are your upcoming meetings for the next 3 days:\n\n1. **wdijwfjw**\n   - Date: 2025-05-11\n   - Time: 21:

('Here are your upcoming meetings for the next 3 days:\n\n1. **wdijwfjw**\n   - Date: 2025-05-11\n   - Time: 21:30 - 22:30 (Europe/Berlin)\n   - Participants: christofbenbach@yahoo.com, marcellusruben@gmail.com\n   - Meeting Link: https://meet.google.com/jey-beio-ugc\n\nPlease let me know if you need further assistance!',
 'b182539a-e26d-4d55-826e-475130bf56ed')

As you can see, the agent called `CalendarEventManagementGetCalendarEventsById` to retrieve the ID of the relevant event from our Google Calendar. Once it had all the necessary information, it called `xpfinish-agent-execution-finished` to signal that it was ready to generate a response for the user.

<a id="Example-Use-Case-3:-Set-Up-Meeting-Recorder"></a>
### Example Use Case 3: Set Up Meeting Recorder

We can also ask our agent to set up a recording bot for a particular meeting with prompt similar to this:

In [7]:
# Create a meeting recorder for a specific meeting
prompt = """
Can you record the onboarding meeting scheduled today? 
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: b1047931-665b-4101-80ed-8a24187dbbd2
Processing Task: 


Can you record the onboarding meeting scheduled today? 



------------------------------------------------------------
AI decided to call :  XpanderMeetingManagementCreateRecordingBot
With the following payload :  {'bodyParams': {'meeting_url': 'https://meet.google.com/ufr-mcmn-kkt', 'bot_name': 'OnboardingRecorder', 'scheduled_time': '2025-04-23T08:00:00+02:00'}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCreateRecordingBot
HTTP Status code :  200
AI decided to call :  XpanderMeetingManagementCheckRecorderStatus
With the following payload :  {'pathParams': {'recorder_id': '0a930729-5661-455e-866f-315297f69832'}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following paylo

('The recording bot has been initiated for the onboarding meeting. The current status is "joining_call". I will check again in a moment to confirm the recording has started.',
 'b1047931-665b-4101-80ed-8a24187dbbd2')

The agent called the `XpanderMeetingManagementCreateRecordingBot` tool to create and set up a recorder for the appropriate meeting. It first fetched the meeting link, then used that link as input when initializing the recording bot. 

<a id="Example-Use-Case-4:-Check-the-Meeting-Recorder-Status"></a>
### Example Use Case 4: Check the Meeting Recorder Status

This time, we’ll ask the agent for the status of the meeting recorder bot it created in the previous interaction. To do this, we’ll use the same thread ID from earlier runs to maintain the agent’s memory and context.

In [10]:
# Check the status of our meeting recorder
prompt = """
Check the status of the meeting recorder I just created for today's meeting.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: b1047931-665b-4101-80ed-8a24187dbbd2
Processing Task: 


Check the status of the meeting recorder I just created for today's meeting.



------------------------------------------------------------
AI decided to call :  XpanderMeetingManagementCheckRecorderStatus
With the following payload :  {'pathParams': {'recorder_id': '0a930729-5661-455e-866f-315297f69832'}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'The recording bot has completed the recording. The video and transcript are available at the following links:\n\n- Video: https://links.xpander.ai/6rs04x1\n- Transcript: https://links.xpander.ai/basalxt\n\nPlease let me know if you need further assistance.', 'is_success': True}}
Executing tool calls
Tool cal

('The recording bot has completed the recording. The video and transcript are available at the following links:\n\n- Video: https://links.xpander.ai/6rs04x1\n- Transcript: https://links.xpander.ai/basalxt\n\nPlease let me know if you need further assistance.',
 'b1047931-665b-4101-80ed-8a24187dbbd2')

To complete the task, the agent called the `XpanderMeetingManagementCheckRecorderStatus` function to retrieve the current status of the recorder bot. Note that to use this function, the agent must provide the recorder bot ID, which it retrieved during the previous interaction.

In the example above, since the recorder bot has finished recording the meeting, the recording assets, such as the video and transcript, are now available for download.



<a id="Example-Use-Case-5:-Send-Recording-Assets-via-Email"></a>
### Example Use Case 5: Send Recording Assets via Email

In the last interaction, the `XpanderMeetingManagementCheckRecorderStatus` returns meeting assets such as the meeting video and transcript after it has finished recording. We can ask the agent to send these assets via email by specifying the email address in our prompt as follows:

In [14]:
prompt = """
Please send the link of the meeting assets to this email address: marcellusruben@gmail.com. 
Write a short and professional letter in the email body with the following format:

Subject: Meeting Assets

Dear <Recipient>,

<email body>

Here are the links to your assets:
- Video: <video link>
- Transcript: <transcript link> 
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: b1047931-665b-4101-80ed-8a24187dbbd2
Processing Task: 


Please send the link of the meeting assets to this email address: marcellusruben@gmail.com. 
Write a short and professional letter in the email body with the following format:

Subject: Meeting Assets

Dear <Recipient>,

<email body>

Here are the links to your assets:
- Video: <video link>
- Transcript: <transcript link> 



------------------------------------------------------------
AI decided to call :  XpanderMessagingServiceSendEmailWithContent
With the following payload :  {'bodyParams': {'to': ['marcellusruben@gmail.com'], 'subject': 'Meeting Assets', 'body_html': '<p>Dear Marcellus,</p><p>Please find the meeting assets below:</p><p>Here are the links to your assets:<br>- Video: <a href="https://links.xpander.ai/6rs04x1">https://links.xpander.ai/6rs04x1</a><br>- Transcript: <a href="https://links.xpander.ai/basalxt">https://lin

('The meeting assets have been sent to marcellusruben@gmail.com with the specified format. Let me know if you need further assistance.',
 'b1047931-665b-4101-80ed-8a24187dbbd2')

As you can see, our agent called the `XpanderMessagingServiceSendEmailWithContent` function to complete the task. If you head over to the email that you specified in the prompt, you'll receive an email from the agent containing the link to the video recording as well as the transcript and the summary of the meeting.

You can also inspect the workflow of your agent from xpander Workbench by clicking on the "Activity" view at the top of the graph visualization of your agent.

![img_7](images/img_7.png)

There you can inspect several types of information via two key components:

- **Thread**: A thread is created every time you assign a task or input prompt to an agent. It includes information such as the total number of tokens generated and the creation timestamp.

- **Log**: This contains a detailed breakdown of your agent's execution steps. You'll see the agent's responses, any tools it called, the payload used for those tool calls, and the responses received after each call.


<a id="Summary:-Working-with-the-Meeting-Recorder-Agent"></a>
## Summary: Working with the Meeting Recorder Agent

In this notebook, we've explored how to use the AI Meeting Recorder Agent for various meeting-related tasks:

1. **Google Calendar Integration**:
   - Authorization and authentication
   - Retrieving upcoming meetings

2. **Meeting Recording Capabilities**:
   - Setting up recording bots for meetings
   - Checking recording status
   - Processing completed recordings

3. **Email Notifications**:
   - Sending meeting summaries via email
   - Including action items and key points

4. **Memory and Context Preservation**:
   - Using thread IDs to maintain conversation context

<a id="Next-Steps"></a>
## Next Steps

In this blueprint, we've created a meeting recorder agent with the help of xpander.ai and NVIDIA NIM. You can now use this blueprint as a starting point to incorporate NVIDIA AI into xpander workflow, or apply it to different use cases that combine xpander and NVIDIA NIM.


We can also further enhance the use of our Meeting Recorder Agent, as examples:

1. Customize the agent with additional tools or capabilities
2. Integrate with other calendar systems or communication platforms
3. Develop more sophisticated workflows for specific meeting types
4. Implement authentication for multiple users

Visit the [Xpander documentation](https://docs.xpander.ai) for more information on extending your agent's capabilities.